In [ ]:
!pip install heapdict
import heapdict as hd
import copy
import numpy as np

### A* Search

This is an implementation of the $A^*$ graph search algorithm to discover the series of moves that transform a moving tile puzzle from an initial state into a desired goal state. For example, for a 3 × 3 puzzle, given the following initial state:

|   |   |   |
|---|---|---|
| 1 | 2 | 3 |
| 4 |   | 6 |
| 7 | 5 | 8 |

and the following goal:

|   |   |   |
|---|---|---|
| 1 | 2 | 3 |
| 4 | 5 | 6 |
| 7 | 8 |   |



#### Implementing Heuristics


#### The number of misplaced tiles.
In the example above, there are two misplaced tiles, 5 and 8.

In [ ]:
def heuristic_misplaced(state):
    """
    The number of misplaced tiles.
    The blank space is not tile and should
    not be included in your misplaced tile count.

    :param state: A tuple of the flattened board.
    :return: The number of misplaced tiles.
    """
    if (type(state) is str):
      return np.Inf

    correct = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    state_arr = np.array(list(state))
    wrongs = state_arr + [0,0,0,0,0,0,0,0,9] - correct
    wrong_count = max(np.count_nonzero(wrongs), 0)
    if(wrongs[-1]!=0):
      wrong_count-=1

    return wrong_count

In [ ]:
def test_heuristic_misplaced():
    res = []
    test_cases = [
    ((1, 2, 3, 4, 0, 6, 7, 5, 8), 2),
    ((1, 2, 3, 4, 5, 6, 7, 8, 0), 0),
    ((1, 2, 3, 4, 5, 6, 7, 0, 8), 1),
    ((1, 2, 3, 4, 5, 6, 0, 7, 8), 2),
    ((1, 2, 3, 4, 0, 5, 7, 8, 6), 2),
]



    for i, (state, expected_value) in enumerate(test_cases):
        actual_value = heuristic_misplaced(state)
        assert actual_value == expected_value, f"Test case for state {state} failed: " \
                                               f"heuristic_misplaced returned {actual_value}, " \
                                               f"expected {expected_value}"

    print("All test cases for heuristic_misplaced passed!")

test_heuristic_misplaced()

All test cases for heuristic_misplaced passed!


#### The sum of the Manhattan distances from the misplaced tiles to their correct positions.

In the example above, the distances from the misplaced tiles 5 and 8 to their correct positions are both 1, so the summed Manhattan distance is 2.

In [ ]:
def heuristic_manhattan(state):
    """
    The sum of the Manhattan distances from the
    misplaced tiles to their correct positions.*

    :param state: A tuple of the flattened board.
    :return: The summed manhattan distances.
    """
    if (type(state) is str):
      return np.Inf

    state_arr = list(state)
    sum = 0
    for j in range(len(state_arr)):
      if j+1 == state_arr[j] or state_arr[j]==0:
        pass
      else:
        row_diff = abs(j+1-state_arr[j]) // 3
        col_diff = abs(j+1-state_arr[j]) % 3
        sum += (row_diff)+(col_diff)
    return sum

In [ ]:
def test_heuristic_manhattan():
    res = []
    test_cases = [
        ((1, 2, 3, 4, 0, 6, 7, 5, 8), 2),
        ((1, 2, 3, 4, 5, 6, 7, 8, 0), 0),
        ((1, 2, 3, 4, 5, 6, 0, 7, 8), 2),
        ((1, 2, 3, 4, 6, 0, 7, 5, 8), 3),
        ((1, 3, 0, 4, 2, 5, 7, 8, 6), 4)
    ]

    for state, expected_value in test_cases:
        actual_value = heuristic_manhattan(state)
        assert actual_value == expected_value, f"Test case for state {state} failed: " \
                                               f"heuristic_manhattan returned {actual_value}, " \
                                               f"expected {expected_value}"

    print("All test cases for heuristic_manhattan passed!")

test_heuristic_manhattan()

All test cases for heuristic_manhattan passed!


##### Implementation of A* search

In [ ]:
def get_moves(state, direction):
  state_dummy = copy.deepcopy(state)
  if direction == 'r':
    if state_dummy.index(0)%3 == 2:
      return 'impossible'
    else:
      i = state_dummy.index(0)
      state_dummy[i] = state_dummy[i+1]
      state_dummy[i+1] = 0
      return state_dummy
  if direction == 'l':
    if state_dummy.index(0)%3 == 0:
      return 'impossible'
    else:
      i = state_dummy.index(0)
      state_dummy[i] = state_dummy[i-1]
      state_dummy[i-1] = 0
      return state_dummy
  if direction == 'u':
    if state_dummy.index(0)//3 == 0:
      return 'impossible'
    else:
      i = state_dummy.index(0)
      state_dummy[i] = state_dummy[i-3]
      state_dummy[i-3] = 0
      return state_dummy
  if direction == 'd':
    if state.index(0)//3 == 2:
      return 'impossible'
    else:
      i = state_dummy.index(0)
      state_dummy[i] = state_dummy[i+3]
      state_dummy[i+3] = 0
      return state_dummy

def isSolvable(arr):
    inv_count = 0
    empty_value = 0
    for i in range(0, 9):
        for j in range(i + 1, 9):
            if arr[j] != empty_value and arr[i] != empty_value and arr[i] > arr[j]:
                inv_count += 1
    return inv_count % 2 == 0

In [ ]:
def astar(init_state, heuristic):
    """
    A^* implementation.

    :param init_state: A tuple of the flattened board.
    :param heuristic: The heuristic function
    :return: A tuple where:
        The first element is a string representing the optimal path.
            Use the characters 'r', 'l', 'u', and 'd' for
            'right', 'left', 'up', and 'down' directions, respectively.
        The second element is a list that contains states in the
            in order they were visited by your algorithm.
    """
    if not isSolvable(init_state):
      return('Not Solvable')

    path = []
    nodes = []
    target_state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
    state = copy.deepcopy(list(init_state))
    states_visited = dict([('', state)])
    h = hd.heapdict()
    if state == target_state:
      path = ['']
      return path[0], [init_state]
    while(state != target_state):
      if len(path)==0:
        h['u'] = (heuristic(get_moves(state, 'u'))+len(path), get_moves(state, 'u'))
        h['d'] = (heuristic(get_moves(state, 'd'))+len(path), get_moves(state, 'd'))
        h['r'] = (heuristic(get_moves(state, 'r'))+len(path), get_moves(state, 'r'))
        h['l'] = (heuristic(get_moves(state, 'l'))+len(path), get_moves(state, 'l'))
      else:
        h[path+'u'] = (heuristic(get_moves(states_visited[path], 'u'))+len(path), get_moves(states_visited[path], 'u'))
        h[path+'d'] = (heuristic(get_moves(states_visited[path], 'd'))+len(path), get_moves(states_visited[path], 'd'))
        h[path+'r'] = (heuristic(get_moves(states_visited[path], 'r'))+len(path), get_moves(states_visited[path], 'r'))
        h[path+'l'] = (heuristic(get_moves(states_visited[path], 'l'))+len(path), get_moves(states_visited[path], 'l'))

      while True:
        best_dir = h.popitem()

        if best_dir[1][1] not in states_visited.values():
          states_visited.update({best_dir[0]: best_dir[1][1]})
          move = best_dir[1][1]
          path = best_dir[0]
          state = move
          break
    return path, [tuple(state) for state in states_visited.values()]

In [ ]:
def test_astar(heuristic):
    test_cases = [
            ((1, 2, 3, 4, 5, 6, 7, 8, 0), "", [(1, 2, 3, 4, 5, 6, 7, 8, 0)]),
            ((1, 2, 3, 4, 5, 6, 0, 7, 8), "rr", [(1, 2, 3, 4, 5, 6, 0, 7, 8),
                                                (1, 2, 3, 4, 5, 6, 7, 0, 8),
                                                (1, 2, 3, 4, 5, 6, 7, 8, 0)]),
            ((1, 2, 3, 4, 6, 0, 7, 5, 8), "ldr", [(1, 2, 3, 4, 6, 0, 7, 5, 8),
                                                (1, 2, 3, 4, 0, 6, 7, 5, 8),
                                                (1, 2, 3, 4, 5, 6, 7, 0, 8),
                                                (1, 2, 3, 4, 5, 6, 7, 8, 0)]),
            ((1, 3, 0, 4, 2, 5, 7, 8, 6), "ldrd", [(1, 3, 0, 4, 2, 5, 7, 8, 6),
                                                (1, 0, 3, 4, 2, 5, 7, 8, 6),
                                                (1, 2, 3, 4, 0, 5, 7, 8, 6),
                                                (1, 2, 3, 4, 5, 0, 7, 8, 6),
                                                (1, 2, 3, 4, 5, 6, 7, 8, 0)]),
        ]

    for test in test_cases:
        state, actual_path, actual_states = test[0], test[1], test[2]
        path, states = astar(test[0], heuristic)
        assert path == actual_path and states == actual_states, f"Test case for state {state} failed: " \
                                                                f"astar returned {path}, {states}, " \
                                                                    f"expected {actual_path, actual_states}"
    print("All test cases for test_astar passed!")

test_astar(heuristic_misplaced)

All test cases for test_astar passed!


#### Check number of nodes opened with different heuristics

Let's now compare the efficienty of different heuristics on different starting states.

In [ ]:
def generate_testcases(number):
  cases = []
  while len(cases) != number:
    test_seq = np.arange(0, 9)
    np.random.shuffle(test_seq)
    if isSolvable(test_seq):
      cases.append(test_seq)
  return cases

In [ ]:
meow = generate_testcases(10)
meow

[array([5, 7, 6, 8, 1, 2, 0, 4, 3]),
 array([1, 4, 5, 7, 3, 2, 0, 6, 8]),
 array([7, 3, 4, 8, 5, 1, 2, 6, 0]),
 array([5, 7, 1, 6, 3, 4, 2, 0, 8]),
 array([8, 4, 5, 1, 0, 2, 7, 6, 3]),
 array([4, 5, 8, 7, 2, 6, 0, 1, 3]),
 array([8, 5, 1, 4, 0, 2, 7, 6, 3]),
 array([3, 5, 7, 1, 0, 6, 2, 8, 4]),
 array([1, 0, 2, 6, 7, 8, 3, 5, 4]),
 array([2, 6, 4, 1, 0, 5, 7, 8, 3])]

In [ ]:
test_cases = [
    ((2, 3, 8, 1, 6, 0, 4, 7, 5)),
    ((1, 3, 0, 4, 2, 5, 7, 8, 6)),
    ((1, 2, 3, 4, 6, 8, 7, 5, 0)),
    ((1, 2, 3, 6, 8, 0, 4, 7, 5)),
    ((4, 1, 2, 6, 0, 3, 7, 5, 8))
    ]

def check_heuristics_difference(test_cases):
  for test in test_cases:
    print(f'manhattan, {test}:   ', astar(test, heuristic_manhattan)[0], '      nodes explored:',len(astar(test, heuristic_manhattan)[1]))
    print(f'misplaced, {test}:   ', astar(test, heuristic_misplaced)[0], '      nodes explored:',len(astar(test, heuristic_misplaced)[1]),'\n')

check_heuristics_difference(test_cases)
check_heuristics_difference(meow)

#Как видно, манхеттенская эвристика из-за того, что она доминимует misplaced-эвристику, изучает либо столько же нодов (для простых кейсов),
#либо меньше нодов (для более сложных кейсов). Эта разница может быть достаточно большой -- например, для кейса [7 3 4 8 5 1 2 6 0]
#это 319 нода для манхэттенской эвристики и 4941 для misplaced эвристики. Как бы то ни было, все они находят одно и то же решение.
#p.s. эта ячейка может выполняться несколько минут

manhattan, (2, 3, 8, 1, 6, 0, 4, 7, 5):    ullddrruldr       nodes explored: 12
misplaced, (2, 3, 8, 1, 6, 0, 4, 7, 5):    ullddrruldr       nodes explored: 36 

manhattan, (1, 3, 0, 4, 2, 5, 7, 8, 6):    ldrd       nodes explored: 5
misplaced, (1, 3, 0, 4, 2, 5, 7, 8, 6):    ldrd       nodes explored: 5 

manhattan, (1, 2, 3, 4, 6, 8, 7, 5, 0):    uldr       nodes explored: 5
misplaced, (1, 2, 3, 4, 6, 8, 7, 5, 0):    uldr       nodes explored: 5 

manhattan, (1, 2, 3, 6, 8, 0, 4, 7, 5):    lldrruldr       nodes explored: 12
misplaced, (1, 2, 3, 6, 8, 0, 4, 7, 5):    lldrruldr       nodes explored: 30 

manhattan, (4, 1, 2, 6, 0, 3, 7, 5, 8):    lurrdldr       nodes explored: 9
misplaced, (4, 1, 2, 6, 0, 3, 7, 5, 8):    lurrdldr       nodes explored: 12 

manhattan, [5 7 6 8 1 2 0 4 3]:    uruldrdlururddluurdldr       nodes explored: 588
misplaced, [5 7 6 8 1 2 0 4 3]:    uruldrdlururddluurdldr       nodes explored: 5482 

manhattan, [1 4 5 7 3 2 0 6 8]:    ruurddllurrdluurdd       no